# Data Cleaning

Organizes CSV files into data frames that are utilized through the rest of the project. Specifically, combines actual stats and projected stats, and creates the necessary columns to explore. Also provides analysis of the starting data frames.

In [1]:
import pandas as pd
from statsmodels.formula.api import ols
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [2]:
actual_stats_2019_df = pd.read_csv('../data/actual_stats_2019.csv')
projected_stats_2019_df = pd.read_csv('../data/projected_stats_2019.csv')

In [3]:
actual_stats_2019_df.head()

,Unnamed: 0,Player,Tm,Pos,Age,G,GS,Cmp,Att,Yds,...,FumblesLost,PassingYds,PassingTD,PassingAtt,RushingYds,RushingTD,RushingAtt,ReceivingYds,ReceivingTD,FantasyPoints
0,0,Christian McCaffrey,CAR,RB,23.0,16.0,16.0,0.0,2.0,0.0,...,0.0,0.0,0.0,2.0,1387.0,15.0,287.0,1005.0,4.0,469.20
1,1,Lamar Jackson,BAL,QB,22.0,15.0,15.0,265.0,401.0,3127.0,...,2.0,3127.0,36.0,401.0,1206.0,7.0,176.0,0.0,0.0,415.68
2,2,Derrick Henry,TEN,RB,25.0,15.0,15.0,0.0,0.0,0.0,...,3.0,0.0,0.0,0.0,1540.0,16.0,303.0,206.0,2.0,294.60
3,3,Aaron Jones,GNB,RB,25.0,16.0,16.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,1084.0,16.0,236.0,474.0,3.0,314.80
4,4,Ezekiel Elliott,DAL,RB,24.0,16.0,16.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,1357.0,12.0,301.0,420.0,2.0,311.70


In [4]:
projected_stats_2019_df.head()

,OVERALL,POS,BYE,BEST,WORST,AVG,STD DEV,ADP,VS. ADP
0,Saquon Barkley,RB1,11.0,1,9,1.7,1.1,1.0,0.0
1,Ezekiel Elliott,RB2,8.0,1,6,2.5,1.3,4.0,2.0
2,Alvin Kamara,RB3,9.0,1,11,3.0,1.1,3.0,0.0
3,Christian McCaffrey,RB4,7.0,1,12,3.1,1.4,2.0,-2.0
4,Nick Chubb,RB5,7.0,2,20,7.5,3.8,10.0,5.0


In [5]:
actual_stats_2019_df.shape

(620, 28)

In [6]:
projected_stats_2019_df.shape

(558, 9)

In [7]:
actual_stats_2019_df.columns

Index(['Unnamed: 0', 'Player', 'Tm', 'Pos', 'Age', 'G', 'GS', 'Cmp', 'Att',
       'Yds', 'Int', 'Att.1', 'Yds.1', 'Tgt', 'Rec', 'Yds.2', 'Y/R', 'Fumbles',
       'FumblesLost', 'PassingYds', 'PassingTD', 'PassingAtt', 'RushingYds',
       'RushingTD', 'RushingAtt', 'ReceivingYds', 'ReceivingTD',
       'FantasyPoints'],
      dtype='object')

In [8]:
as_2019_df = actual_stats_2019_df[["Player", "Pos", "FantasyPoints"]]
ps_2019_df = projected_stats_2019_df[["OVERALL", "POS", "ADP"]]
ps_2019_df = ps_2019_df.rename(columns = {"OVERALL": "Player", "POS": "ExpertRanking"})

In [9]:
as_2019_df.shape

(620, 3)

In [10]:
ps_2019_df.shape

(558, 3)

In [11]:
as_2019_df = as_2019_df.sort_values(by=['FantasyPoints'], ascending=False)
as_2019_df

,Player,Pos,FantasyPoints
0,Christian McCaffrey,RB,469.20
1,Lamar Jackson,QB,415.68
6,Michael Thomas,WR,374.60
11,Dak Prescott,QB,335.78
13,Russell Wilson,QB,326.60
...,...,...,...
612,Sean Mannion,QB,-1.46
617,Jarrett Stidham,QB,-1.64
613,Greg Dortch,0,-2.00
615,Ray-Ray McCloud,0,-2.00


# Dataframe analysis

The dataframe representing the actual stats originally contained 620 rows and 28 columns. Many of these columns provided very detailed stats which are not necessary for my analysis. I reduced this to 3 columns. The ordering of the rows are based on the fantasy points.

The projected stats dataframe has 558 rows and 9 columns. Similarly I reduced this dataframe down to 3 columns. The Pos column represents the ranking by position according to many experts. The ADP column represents the average draft pick by all fantasy football players. In both dataframes, the rows are individual NFL players.

In [12]:
#Merges both data frames

merged_2019_df = ps_2019_df.merge(as_2019_df, how='inner', on = ['Player'])

In [13]:
#Removes position from expert ranking and casts to int

merged_2019_df["ExpertRanking"] = merged_2019_df["ExpertRanking"].str[2:].astype(int)

In [14]:
#Adds ADP_rerank column which ranks ADP (average draft pick) by position

pos_groups=merged_2019_df.sort_values('ADP').groupby('Pos').transform(lambda v: list(range(1,v.shape[0]+1)))
merged_2019_df['ADP_rerank']=pos_groups['ADP']

In [15]:
merged_2019_df

,Player,ExpertRanking,ADP,Pos,FantasyPoints,ADP_rerank
0,Saquon Barkley,1,1.0,RB,244.10,1
1,Ezekiel Elliott,2,4.0,RB,311.70,4
2,Alvin Kamara,3,3.0,RB,248.52,3
3,Christian McCaffrey,4,2.0,RB,469.20,2
4,Nick Chubb,5,10.0,RB,255.20,8
...,...,...,...,...,...,...
409,Durham Smythe,101,NaN,TE,13.50,82
410,Antony Auclair,102,NaN,TE,2.10,83
411,Johnny Mundt,103,NaN,TE,6.60,84
412,Matt Moore,58,NaN,QB,42.26,55


In [16]:
# save the subset and merged data frame for use in subsequent analysis 

merged_2019_df.to_csv('../data/cleaned/actual_and_projected_2019.csv', index=False)

In [17]:
#Adds year for 2019 dataframe

merged_2019_df["Year"] = "2019"

In [18]:
# Repeat process for 2018

actual_stats_2018_df = pd.read_csv('../data/actual_stats_2018.csv')
projected_stats_2018_df = pd.read_csv('../data/projected_stats_2018.csv')

as_2018_df = actual_stats_2018_df[["Player", "Pos", "FantasyPoints"]]
ps_2018_df = projected_stats_2018_df[["OVERALL", "POS", "ADP"]]
ps_2018_df = ps_2018_df.rename(columns = {"OVERALL": "Player", "POS": "ExpertRanking"})

as_2018_df = as_2018_df.sort_values(by=['FantasyPoints'], ascending=False)

merged_2018_df = ps_2018_df.merge(as_2018_df, how='inner', on = ['Player'])
merged_2018_df["ExpertRanking"] = merged_2018_df["ExpertRanking"].str[2:].astype(int)
pos_groups=merged_2018_df.sort_values('ADP').groupby('Pos').transform(lambda v: list(range(1,v.shape[0]+1)))
merged_2018_df['ADP_rerank']=pos_groups['ADP']
merged_2018_df["Year"] = "2018"

In [19]:
# Repeat process for 2017

actual_stats_2017_df = pd.read_csv('../data/actual_stats_2017.csv')
projected_stats_2017_df = pd.read_csv('../data/projected_stats_2017.csv')

as_2017_df = actual_stats_2017_df[["Player", "Pos", "FantasyPoints"]]
ps_2017_df = projected_stats_2017_df[["OVERALL", "POS", "ADP"]]
ps_2017_df = ps_2017_df.rename(columns = {"OVERALL": "Player", "POS": "ExpertRanking"})

as_2017_df = as_2017_df.sort_values(by=['FantasyPoints'], ascending=False)

merged_2017_df = ps_2017_df.merge(as_2017_df, how='inner', on = ['Player'])
merged_2017_df["ExpertRanking"] = merged_2017_df["ExpertRanking"].str[2:].astype(int)
pos_groups=merged_2017_df.sort_values('ADP').groupby('Pos').transform(lambda v: list(range(1,v.shape[0]+1)))
merged_2017_df['ADP_rerank']=pos_groups['ADP']
merged_2017_df["Year"] = "2017"

In [20]:
# Repeat process for 2016

actual_stats_2016_df = pd.read_csv('../data/actual_stats_2016.csv')
projected_stats_2016_df = pd.read_csv('../data/projected_stats_2016.csv')

as_2016_df = actual_stats_2016_df[["Player", "Pos", "FantasyPoints"]]
ps_2016_df = projected_stats_2016_df[["OVERALL", "POS", "ADP"]]
ps_2016_df = ps_2016_df.rename(columns = {"OVERALL": "Player", "POS": "ExpertRanking"})

as_2016_df = as_2016_df.sort_values(by=['FantasyPoints'], ascending=False)

merged_2016_df = ps_2016_df.merge(as_2016_df, how='inner', on = ['Player'])
merged_2016_df["ExpertRanking"] = merged_2016_df["ExpertRanking"].str[2:].astype(int)
pos_groups=merged_2016_df.sort_values('ADP').groupby('Pos').transform(lambda v: list(range(1,v.shape[0]+1)))
merged_2016_df['ADP_rerank']=pos_groups['ADP']
merged_2016_df["Year"] = "2016"

In [21]:
# Repeat process for 2015

actual_stats_2015_df = pd.read_csv('../data/actual_stats_2015.csv')
projected_stats_2015_df = pd.read_csv('../data/projected_stats_2015.csv')

as_2015_df = actual_stats_2015_df[["Player", "Pos", "FantasyPoints"]]
ps_2015_df = projected_stats_2015_df[["OVERALL", "POS", "ADP"]]
ps_2015_df = ps_2015_df.rename(columns = {"OVERALL": "Player", "POS": "ExpertRanking"})

as_2015_df = as_2015_df.sort_values(by=['FantasyPoints'], ascending=False)

merged_2015_df = ps_2015_df.merge(as_2015_df, how='inner', on = ['Player'])
merged_2015_df["ExpertRanking"] = merged_2015_df["ExpertRanking"].str[2:].astype(int)
pos_groups=merged_2015_df.sort_values('ADP').groupby('Pos').transform(lambda v: list(range(1,v.shape[0]+1)))
merged_2015_df['ADP_rerank']=pos_groups['ADP']
merged_2015_df["Year"] = "2015"

In [22]:
#Combines all years into one dataframe

merged_df = pd.concat([merged_2015_df, merged_2016_df, merged_2017_df, merged_2018_df, merged_2019_df])
merged_df

,Player,ExpertRanking,ADP,Pos,FantasyPoints,ADP_rerank,Year
0,Le'Veon Bell,1,1.0,RB,111.20,1,2015
1,Adrian Peterson,2,2.0,RB,260.70,2,2015
2,Jamaal Charles,3,3.0,RB,101.10,3,2015
3,Antonio Brown,1,4.0,WR,378.20,1,2015
4,Eddie Lacy,4,5.0,RB,140.60,4,2015
...,...,...,...,...,...,...,...
409,Durham Smythe,101,NaN,TE,13.50,82,2019
410,Antony Auclair,102,NaN,TE,2.10,83,2019
411,Johnny Mundt,103,NaN,TE,6.60,84,2019
412,Matt Moore,58,NaN,QB,42.26,55,2019


In [23]:
# save the subset and merged data frame for use in subsequent analysis for all years 

merged_df.to_csv('../data/cleaned/actual_and_projected.csv', index=False)